# Keras Tuner 2024

to test and search for the best architecture and hyper parameter combination

In [1]:
from tensorflow import keras
import numpy as np
import keras_tuner

from tensorflow import keras
from tensorflow.keras import layers

# Load the data

In [2]:
(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()


# Split the data

In [3]:
x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

# Reshaping and normalization

In [4]:
x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

# Output Layer preparing

In [5]:
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Build the model

### 1. With static count of hidden layers -> 1x hidden layer with dyanmic count of neurons

In [6]:
def build_model(hp):

    model = keras.Sequential()
    model.add(layers.Flatten())

    model.add(
        layers.Dense(
            units=hp.Int("units", min_value=32, max_value=512,step=32), # tune the count of neorons

            activation=hp.Choice("activation", ["relu", "tanh"]) # tune the activation function

        ))



    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))

    model.add(layers.Dense(10,activation="softmax"))  # Output Layer

    # Dynamic LR
    learning_rate  =  hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")


    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate),
        loss = "categorical_crossentropy",
        metrics=["accuracy"]
    )




    return model

### 2. With dynamic count of hidden layers

In [7]:
def build_model(hp):

    model = keras.Sequential()
    model.add(layers.Flatten())


    # tune the counf of the hidden layers
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                units=hp.Int(f"units_{i}", min_value=32, max_value=512,step=32), # tune the count of neorons

                activation=hp.Choice("activation", ["relu", "tanh"]) # tune the activation function

            ))



    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))

    model.add(layers.Dense(10,activation="softmax"))  # Output Layer

    # Dynamic LR
    learning_rate  =  hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")


    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate),
        loss = "categorical_crossentropy",
        metrics=["accuracy"]
    )




    return model

# Test if the build_model working with hp

In [8]:
hp = keras_tuner.HyperParameters()

build_model(hp)

 

# Build the tuner

In [9]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=2, # combinatin of params
    executions_per_trial=2,
    overwrite=True,
    directory="keras_tuner",
    project_name="MyTuner1"
)

In [10]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [11]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Trial 2 Complete [00h 01m 15s]
val_accuracy: 0.9754500091075897

Best val_accuracy So Far: 0.9754500091075897
Total elapsed time: 00h 01m 55s


# Get Result Summary

In [12]:
tuner.results_summary()

Results summary
Results in keras_tuner\MyTuner1
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
num_layers: 3
units_0: 352
activation: relu
dropout: False
lr: 0.0014835474564790457
units_1: 96
units_2: 32
Score: 0.9754500091075897

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 32
activation: tanh
dropout: True
lr: 0.0008711452772441899
units_1: 32
Score: 0.9625999927520752


# Get the best Hyper Paramter to retrain the model

In [13]:
best_hps  = tuner.get_best_hyperparameters(3) # get a list of top 3 hp
best_hps  = tuner.get_best_hyperparameters() # get a list of top 1 hp

model = build_model(best_hps[0]) # [0]get the top best hp and build the model

# Fit the model with the entire dataset
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))

model.fit(x=x_all, y=y_all, epochs= 1)



1875/1875 [==============================] - 9s 4ms/step - loss: 0.2141 - accuracy: 0.9359
